In [3]:
from rdkit import Chem
from rdkit.Chem import AllChem
import rdkit.Chem.rdMolDescriptors as d
import rdkit.Chem.Fragments as f
import rdkit.Chem.Lipinski as l
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [50]:
m = Chem.MolFromSmiles('Cc1ccccc1')
m.GetNumAtoms()
# rdMolDescriptors
d.CalcExactMolWt(m)
# Fragments
f.fr_Al_COO(m)
# Lipinski 
l.HeavyAtomCount(m)
# Fingerprints
fp = AllChem.GetMorganFingerprintAsBitVect(m,2,nBits=124)
np.array(fp)

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [58]:
data = pd.read_csv('training_smiles.csv')
data.head()

,INDEX,SMILES,ACTIVE
0,1,CC(C)N1CC(=O)C(c2nc3ccccc3[nH]2)=C1N,0.0
1,2,COc1ccc(-c2ccc3c(N)c(C(=O)c4ccc(OC)c(OC)c4)sc3...,0.0
2,3,CCc1ccc(C(=O)COC(=O)CCc2nc(=O)c3ccccc3[nH]2)cc1,0.0
3,4,O=C(CN1CCOCC1)Nc1ccc(S(=O)(=O)N2CCCCCC2)cc1,0.0
4,5,C=CCC(Nc1ccccc1)c1ccc(OC)c(OC)c1,0.0


In [59]:
df = data.copy()
df['mol'] = df['SMILES'].apply(Chem.MolFromSmiles)
# df['molwt'] = df['mol'].apply(d.CalcExactMolWt)
# df['fr_Al_COO'] = df['mol'].apply(f.fr_Al_COO)
# df['HeavyAtomCount'] = df['mol'].apply(l.HeavyAtomCount)
# df['fp'] = df['mol'].apply(lambda x: AllChem.GetMorganFingerprintAsBitVect(x,2,nBits=124))

RDKit WARNING: [13:32:08] WARNING: not removing hydrogen atom without neighbors
[13:32:08] WARNING: not removing hydrogen atom without neighbors


### FEATURES WITHIN EACH ATOM
https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-018-2523-5/tables/1

In [ ]:
# df['most_common_atom'] = df['mol'].apply(lambda x: x.Chem.GetAtomWithIdx(x.GetMostCommonAtomIdx()).GetSymbol())
df['number_atoms'] = df['mol'].apply(Chem.GetNumAtoms)
df['number_hydrogen'] = df['mol'].GetNumHeavyAtoms()
df['unsaturation'] = df['mol'].GetNumHeteroatoms()
df['formal_charge'] = df['mol'].GetFormalCharge()
df['total_valence'] = df['mol'].GetTotalValence()
df['ring'] = df['mol'].IsInRing()
df['aromatic_atoms'] = df['mol'].GetAromaticAtoms()
df['chirality_atoms'] = df['mol'].GetChiralAtoms()
df['hybridization_atoms'] = df['mol'].GetHybridizationAtoms()

df.to_csv('training_processed_2.csv', index=False)

### FEATURES WITHIN EACH SMILE

https://aip.scitation.org/doi/pdf/10.1063/1.5062773 --> 11 features extracted out of the number of elements of a certain type in the bond

In [53]:
df['smiles_length'] = df['SMILES'].apply(len)
df['number_atoms'] = df['mol'].apply(lambda x: x.GetNumAtoms()) # number of atoms in the molecule
df['number_bonds'] = df['mol'].apply(lambda x: x.GetNumBonds())
df['number_carbon_atoms'] = df['mol'].apply(lambda x: len([atom for atom in x.GetAtoms() if atom.GetAtomicNum() == 6]))
df['number_nitrogen_atoms'] = df['mol'].apply(lambda x: len([atom for atom in x.GetAtoms() if atom.GetAtomicNum() == 7]))
df['number_potasium_atoms'] = df['mol'].apply(lambda x: len([atom for atom in x.GetAtoms() if atom.GetAtomicNum() == 19]))
df['number_sulfur_atoms'] = df['mol'].apply(lambda x: len([atom for atom in x.GetAtoms() if atom.GetAtomicNum() == 16]))
df['number_clorine_atoms'] = df['mol'].apply(lambda x: len([atom for atom in x.GetAtoms() if atom.GetAtomicNum() == 17]))
df['number_bromine_atoms'] = df['mol'].apply(lambda x: len([atom for atom in x.GetAtoms() if atom.GetAtomicNum() == 35]))
df['number_iodine_atoms'] = df['mol'].apply(lambda x: len([atom for atom in x.GetAtoms() if atom.GetAtomicNum() == 53]))
df['number_oxygen_atoms'] = df['mol'].apply(lambda x: len([atom for atom in x.GetAtoms() if atom.GetAtomicNum() == 8]))

# --- not included in paper
df['number_hydrogen_atoms'] = df['mol'].apply(lambda x: len([atom for atom in x.GetAtoms() if atom.GetAtomicNum() == 1]))
df['number_fluorine_atoms'] = df['mol'].apply(lambda x: len([atom for atom in x.GetAtoms() if atom.GetAtomicNum() == 9]))


df.to_csv('training_preprocessed.csv', index=False)

In [6]:
# Add functions from previous assignments
import functions as f
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import preprocessing

# get the features and labels
data = pd.read_csv('training_preprocessed.csv')
features = data.drop(['SMILES', 'mol', 'ACTIVE', 'INDEX'], axis=1)
labels = data['ACTIVE']

features_processed = features.loc[:, (features != 0).any(axis=0)] # drop columns with all zeros
features_processed = features_processed.dropna(axis=1, how='all') # drop columns with all Nan

mm = preprocessing.MinMaxScaler()
features_processed = pd.DataFrame(mm.fit_transform(features_processed))

# Apply PCA after scaling the data
features_processed = StandardScaler().fit_transform(features_processed)
pca = PCA(n_components=0.95) #scikit-learn choose the minimum number of principal components such that 95% of the variance is retained
features_pca = pca.fit_transform(features_processed)


# add into csv features and labels
training_processed = pd.DataFrame(features_pca)
# add title to all features
training_processed.columns = ['feature_' + str(i) for i in range(1, len(training_processed.columns)+1)]
training_processed['ACTIVE'] = labels
training_processed.to_csv('training_processed.csv', index=False)

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

logisticRegr = LogisticRegression(solver = 'lbfgs')

# split training data into training and validation 
data_train, data_valid, labels_train, labels_valid = train_test_split(features_pca, labels, test_size=0.2, random_state=30) # change random_state to get different results

logisticRegr.fit(data_train, labels_train)

logisticRegr.predict(data_valid)
logisticRegr.score(data_valid, labels_valid)

0.9892166901318316